In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import time
import random
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [ ]:
import os
import sys
sys.path.append('../')

In [ ]:
from model import UnicycleModel
from cost import UnicycleCost
from constraints import UnicycleConstraints
from scipy.integrate import solve_ivp
from matplotlib.patches import Ellipse
from utils.utils_alg import get_neighbor_vec,get_K_discrete
from utils.utils_plot import plot_traj,plot_traj_set,plot_state_input

from trajopt import trajopt
from funlopt import funlopt
from Lipschitz import Lipschitz
from Scaling import TrajectoryScaling

from utils.utils_alg import get_sample_eta_w,propagate_model
import cvxpy as cvx
from scipy.integrate import solve_ivp



In [ ]:
N = 30
tf = 5
delT = tf/N

## obstacle setting

In [ ]:
def get_H_obs(rx,ry) :
    return np.diag([1/rx,1/ry])
# obstacle
c_list = []
H_list = []


c_list.append([1.8,0.3])
H_list.append(get_H_obs(0.5,0.5))

c_list.append([2*2/3,3.5])
H_list.append(get_H_obs(0.5,0.5))

c_list.append([4.2,0.7])
H_list.append(get_H_obs(0.5,0.5))
c_list.append([3.8,3.5])
H_list.append(get_H_obs(0.5,0.5))

c_list.append([10*2/3,0.5])
H_list.append(get_H_obs(0.5,0.5))
c_list.append([6.5,4])
H_list.append(get_H_obs(0.5,0.5))



c_list.append([4*2/3,1.7])
H_list.append(get_H_obs(0.5,0.5))
c_list.append([8*2/3,2.3])
H_list.append(get_H_obs(0.5,0.5))

c_list.append([0.5,2])
H_list.append(get_H_obs(0.5,0.5))
c_list.append([8,2.3])
H_list.append(get_H_obs(0.5,0.5))


## initial and final conditions 

In [ ]:
myModel = UnicycleModel.unicycle2('unicycle','numeric_central')

ix = myModel.ix
iu = myModel.iu
iw = myModel.iw
iq = myModel.iq
ip = myModel.ip

C = myModel.C
D = myModel.D
E = myModel.E
G = myModel.G

myCost = UnicycleCost.unicycle('Hello',ix,iu,N)
myConst = UnicycleConstraints.UnicycleConstraints('Hello',ix,iu)
myConst.set_obstacle(c_list,H_list)



In [ ]:
xi = np.zeros(3)
xi[0] = 0.0
xi[1] = 0.0 
xi[2] = 0

xf = np.zeros(3)
xf[0] = 8.0
xf[1] = 4.0
xf[2] = 0

Qini = np.diag([0.4**2,0.4**2,np.deg2rad(20)**2])
Qf = np.diag([0.3**2,0.3**2,np.deg2rad(20)**2])

x0 = np.zeros((N+1,ix))
for i in range(N+1) :
    x0[i] = (N-i)/N * xi + i/N * xf
u0 = np.zeros((N+1,iu))

In [ ]:
A,B,s,z,x_prop_n = myModel.diff_discrete_zoh(x0[0:N,:],u0[0:N,:],delT,tf) 
S = np.eye(ix)
R = 1*np.eye(iu)
K0 = get_K_discrete(A,B,S,R,S,N,ix,iu)
Q0 = np.tile(np.diag([0.35**2,0.35**2,np.deg2rad(10)**2]),(N+1,1,1))
Y0 = K0@Q0[:N]
betahat = np.ones(N+1)

In [ ]:
x_max = np.array([10,10,np.pi])
x_min = np.zeros(ix)

u_max = np.array([5,5]) 
u_min = np.zeros(iu)

traj_scaling = TrajectoryScaling(x_min,x_max,u_min,u_max,tf)

x_max = np.array([1,1,np.pi])
x_min = np.zeros(ix)

u_max = np.array([5,5]) 
u_min = np.zeros(iu)

funl_scaling = TrajectoryScaling(x_min,x_max,u_min,u_max,tf)

In [ ]:
traj_solver= trajopt('unicycle',N,tf,10,myModel,myCost,myConst,Scaling=traj_scaling,
              w_c=1,w_vc=1e3,w_tr=1e0,tol_vc=1e-6,tol_tr=1e-4,tol_dyn=1e-3,verbosity=True)

In [ ]:
_,_,xnom,unom,_,total_num_iter,_,_,_,_,history_nom = traj_solver.run(x0,u0,xi,xf)
tnom = np.array(range(N+1))*delT

In [ ]:
fS = 15
plt.figure(0,figsize=(15,7))
plot_traj_set(x0,u0,c_list,H_list,Q0,xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt)
plt.plot(xnom[:,0],xnom[:,1],'o-',color='tab:brown',label='traj w/o funnel')
plt.axis([-1.0, 9.0, -1.0, 5.0])
plt.figure(1,figsize=(10,4))
plt.subplot(121)
plt.plot(tnom, unom[:,0]*0+myConst.vmax,'-.',color='tab:red',alpha=1.0,linewidth=2.0,label='limit')
plt.plot(tnom, unom[:,0]*0+myConst.vmin,'-.',color='tab:red',alpha=1.0,linewidth=2.0)
plt.step(tnom, [*unom[:N,0],unom[N-1,0]],'-',color='tab:brown',alpha=1.0,where='post',linewidth=2.0,label='nominal')
plt.plot(1e3, 1e3,'-',color='tab:purple',alpha=1.0,linewidth=1.0,label='samples')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('$u_v$ (m/s)', fontsize = fS)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.axis([0.0, tf, -0.5, 3.5])
plt.grid(True)

plt.subplot(122)
plt.plot(tnom, unom[:,1]*0+myConst.wmax,'-.',color='tab:red',alpha=1.0,linewidth=2.0)
plt.plot(tnom, unom[:,1]*0+myConst.wmin,'-.',color='tab:red',alpha=1.0,linewidth=2.0,label='limit')
plt.step(tnom, [*unom[:N,1],unom[N-1,1]],'-',color='tab:brown',alpha=1.0,where='post',linewidth=2.0,label='nominal')
plt.plot(1e3, 1e3,'-',color='tab:purple',alpha=1.0,linewidth=1.0,label='samples')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('$u_{\Theta}$ (rad/s)', fontsize = fS)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.axis([0.0, tf, -3, 3])
plt.legend(fontsize=fS)
plt.rcParams["font.family"] = "Times New Roman"
plt.grid(True)

## run

In [ ]:
total_iter = 20

tol_funl = 1e-8
tol_traj = 1e-8
tol_vc = 1e-8
tol_dyn = 1e-8

traj_solver= trajopt('unicycle',N,tf,10,myModel,myCost,myConst,Scaling=traj_scaling,
              w_c=1,w_vc=1e3,w_tr=1e0,tol_vc=1e-6,tol_tr=1e-4,tol_dyn=1e-3,verbosity=False)

funl_solver = funlopt(ix,iu,iq,ip,iw,N,myCost.S,myCost.R,funl_scaling,
                      alpha=0.99,
                      lambda_mu=0.1,
                      w_tr=1e-4,
                      flag_nonlinearity=True)
funl_solver.cvx_initialize(Qini,Qf)
lip_estimator = Lipschitz(ix,iu,iq,ip,iw,N,num_sample=100,flag_uniform=True)

from jointsynthesis import jointsynthesis
JS = jointsynthesis(myModel,traj_solver,funl_solver,lip_estimator,total_iter,
                   tol_traj,tol_funl,tol_vc,tol_dyn,verbosity=True)

In [ ]:
history = JS.run(xi,xf,x0,u0,Q0,Y0,K0)

In [ ]:
del_traj_list = [history[i]['traj_diff'] for i in range(len(history))]
del_funl_list = [history[i]['funl_diff'] for i in range(len(history))]
vc_list = [history[i]['vc'] for i in range(len(history))]
dyn_error_list = [history[i]['dyn_error'] for i in range(len(history))]

In [ ]:
i_index = np.array([i+1 for i in range(len(history))])
fS = 20
plt.figure(figsize=(15,8))
plt.subplot(221)
plt.plot(i_index,vc_list,'o-')
plt.plot(i_index,i_index*0+tol_vc,'--')
# plt.xlabel('iteration number',fontsize=fS,fontname='Times New Roman')
plt.ylabel(r'$\Delta_{vc}$',fontsize=fS,fontname='Times New Roman')
plt.yscale('log')
# plt.ylim([10**(-6), 10**(4)])
plt.grid(True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.subplot(222)
plt.plot(i_index,dyn_error_list,'o-')
plt.plot(i_index,i_index*0+tol_dyn,'--',label='tolerance')
# plt.xlabel('iteration number',fontsize=fS,fontname='Times New Roman')
plt.ylabel(r'$\Delta_{dyn}$',fontsize=fS,fontname='Times New Roman')
plt.yscale('log')
# plt.ylim([10**(-6), 10**(4)])
plt.grid(True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=fS)


plt.subplot(223)
plt.plot(i_index,del_traj_list,'o-')
plt.plot(i_index,i_index*0+tol_traj,'--')
plt.xlabel('iteration number',fontsize=fS,fontname='Times New Roman')
plt.ylabel(r'$\Delta_T$',fontsize=fS,fontname='Times New Roman')
plt.yscale('log')
# plt.ylim([10**(-6), 10**(4)])
plt.grid(True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.subplot(224)
plt.plot(i_index,del_funl_list,'o-')
plt.plot(i_index,i_index*0+tol_funl,'--')
plt.xlabel('iteration number',fontsize=fS,fontname='Times New Roman')
plt.ylabel(r'$\Delta_F$',fontsize=fS,fontname='Times New Roman')
plt.yscale('log')
# plt.ylim([10**(-6), 10**(4)])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.rcParams["font.family"] = "Times New Roman"
plt.grid(True)


# plt.savefig('../images/convergence_result.pdf')

# Computational time

In [ ]:
time_derivs = []
time_trajopt = []
time_funlopt = []
time_lipschitz = []

for his in history :
    time_derivs.append(his['t_derivs'])
    time_trajopt.append(his['t_trajopt'])
    time_funlopt.append(his['t_funlopt'])
    time_lipschitz.append(his['t_Lipschitz'])
def mean_list(l) :
    return sum(l)/len(l)
mean_derivs = mean_list(time_derivs)
mean_trajopt = mean_list(time_trajopt)
mean_funlopt = mean_list(time_funlopt)
mean_lipschitz = mean_list(time_lipschitz)
print("mean_derivs",mean_derivs)
print("mean_trajopt",mean_trajopt)
print("mean_funlopt",mean_funlopt)
print("mean_lipschitz",mean_lipschitz)

# Evaluate the generated traj and funl

In [ ]:
from utils.utils_alg import forward_full_with_K,get_sample_trajectory
import scipy

In [ ]:
xbar = history[-1]['x']
ubar = history[-1]['u']
Qbar = history[-1]['Q']
Ybar = history[-1]['Y']
Kbar = history[-1]['K']
gammabar = history[-1]['gamma']

# Sample from initial set

In [ ]:
x0_sample = []
num_sample = 5
for i in range(num_sample) :
    z = np.random.randn(ix)
    z = z / np.linalg.norm(z)
    z = np.array([0,1,0])
    x_s = xbar[0] + scipy.linalg.sqrtm(Qbar[0])@z
    x0_sample.append(x_s)
    



In [ ]:
tsam,xsam,usam,wsam,xsamp = get_sample_trajectory(xi,x0_sample,xbar,ubar,Qbar,Ybar,myModel,N,ix,iu,iw,delT,
                                            flag_noise=True,discrete=True)


In [ ]:
fS = 15
plt.figure(0,figsize=(15,7))
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt)
plt.legend(fontsize=fS)
plt.rcParams["font.family"] = "Times New Roman"
plt.axis([-1.0, 9.0, -1.0, 5.0])
# plt.savefig('../images/first_iteration/proposed.pdf')

In [ ]:
fS = 20
plt.figure(0,figsize=(15,7))
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt,flag_label=True,fS=fS)
# for xsam_e in xsam :
for xsam_e in xsamp :
    plt.plot(xsam_e[:,0], xsam_e[:,1],'o-',markersize=3.0, linewidth=1.0,alpha=1.0,color='tab:purple')
plt.plot(1e3,1e3,'o-',color='tab:purple',label='samples')
# plt.plot(xnom[:,0],xnom[:,1],'-.',color='tab:brown',label='traj w/o funnel')
plt.legend(fontsize=fS)
plt.grid(True)
plt.rcParams["font.family"] = "Times New Roman"
plt.axis([-1.0, 9.0, -1.0, 5.0])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
# plt.savefig('../images/traj_result_new.pdf')

In [ ]:
fS = 20
plt.figure(0,figsize=(15,5))
alpha = 1.0
t_index = np.array(range(N+1))*delT
for i in range(num_sample) :
    tsam_e = tsam[i]
    xsam_e = xsam[i]
    usam_e = usam[i]
#     plot_state_input(tsam_e,xsam_e,usam_e,None,None,N,delT,alpha,plt,flag_step=False)

    plt.subplot(121)
    plt.plot(tsam_e, usam_e[:,0],color='tab:purple',alpha=alpha,linewidth=2.0)
    plt.subplot(122)
    plt.plot(tsam_e, usam_e[:,1],color='tab:purple',alpha=alpha,linewidth=2.0)
plt.subplot(121)
plt.plot(tsam_e, usam_e[:,0]*0+myConst.vmax,'-.',color='tab:red',alpha=1.0,linewidth=2.0,label='limit')
plt.plot(tsam_e, usam_e[:,0]*0+myConst.vmin,'-.',color='tab:red',alpha=1.0,linewidth=2.0)
plt.step(t_index, [*ubar[:N,0],ubar[N-1,0]],'--',color='tab:orange',alpha=1.0,where='post',linewidth=2.0,label='nominal')
plt.plot(1e3, 1e3,'-',color='tab:purple',alpha=1.0,linewidth=1.0,label='samples')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('$u_v$ (m/s)', fontsize = fS)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.axis([0.0, tf, -0.5, 3.5])
plt.grid(True)
plt.subplot(122)
plt.plot(tsam_e, usam_e[:,1]*0+myConst.wmax,'-.',color='tab:red',alpha=1.0,linewidth=2.0)
plt.plot(tsam_e, usam_e[:,1]*0+myConst.wmin,'-.',color='tab:red',alpha=1.0,linewidth=2.0,label='limit')
plt.step(t_index, [*ubar[:N,1],ubar[N-1,1]],'--',color='tab:orange',alpha=1.0,where='post',linewidth=2.0,label='nominal')
plt.plot(1e3, 1e3,'-',color='tab:purple',alpha=1.0,linewidth=1.0,label='samples')
plt.xlabel('time (s)', fontsize = fS)
plt.ylabel('$u_{\Theta}$ (rad/s)', fontsize = fS)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.axis([0.0, tf, -3, 3])
plt.legend(fontsize=fS)
plt.rcParams["font.family"] = "Times New Roman"
plt.grid(True)
# plt.savefig('../images/input_result_new.pdf')


In [ ]:
# fS = 20
# plt.figure(0,figsize=(10,5))
# alpha = 0.5
# t_index = np.array(range(N+1))*delT
# for i in range(num_sample) :
#     tsam_e = tsam[i]
#     xsam_e = xsam[i]
#     wsam_e = wsam[i]
# #     plot_state_input(tsam_e,xsam_e,usam_e,None,None,N,delT,alpha,plt,flag_step=False)
#     plt.subplot(121)
#     plt.plot(tsam_e, wsam_e[:,0],color='tab:purple',alpha=alpha,linewidth=1.0)
#     plt.subplot(122)
#     plt.plot(tsam_e, wsam_e[:,1],color='tab:purple',alpha=alpha,linewidth=1.0)
# plt.subplot(121)
# plt.plot(1e3, 1e3,'-',color='tab:purple',alpha=1.0,linewidth=1.0,label='samples')
# plt.xlabel('time (s)', fontsize = fS)
# plt.ylabel('w1 (m)', fontsize = fS)
# plt.axis([0.0, tf, -2, 2])
# plt.subplot(122)
# plt.plot(1e3, 1e3,'-',color='tab:purple',alpha=1.0,linewidth=1.0,label='samples')
# plt.xlabel('time (s)', fontsize = fS)
# plt.ylabel('w2 (m)', fontsize = fS)
# plt.axis([0.0, tf, -2, 2])
# plt.legend(fontsize=fS)
# plt.rcParams["font.family"] = "Times New Roman"
# # plt.savefig('../images/disturbance.pdf')

In [ ]:
fS = 15
plt.figure(0,figsize=(4,4))
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt,flag_label=False)
for xsam_e in xsamp :
    plt.plot(xsam_e[:,0], xsam_e[:,1],'o',markersize=1.0, linewidth=1.0,alpha=1.0,color='tab:purple')
plt.plot(1e3,1e3,'o-',color='tab:purple',label='samples')
# plt.plot(xnom[:,0],xnom[:,1],'-.',color='tab:brown',label='traj w/o funnel')
# plt.legend(fontsize=fS)
plt.grid(True)
plt.rcParams["font.family"] = "Times New Roman"
plt.axis([3.7, 5.7, 1.5, 3.5])
plt.axis('off')
# plt.savefig('../images/traj_result_new_expansion.pdf')

In [ ]:
fS = 15
plt.figure(0,figsize=(15,7))
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt,flag_label=True)
for xsam_e in xsamp :
    plt.plot(xsam_e[:,0], xsam_e[:,1],'o',markersize=3.0, linewidth=1.0,alpha=1.0,color='tab:purple')
#     plt.plot(xsam_e[1,0], xsam_e[1,1],'o',markersize=3.0, linewidth=1.0,alpha=1.0,color='tab:purple')
plt.plot(1e3,1e3,'o-',color='tab:purple',label='samples')
# plt.plot(xnom[:,0],xnom[:,1],'-.',color='tab:brown',label='traj w/o funnel')
# plt.legend(fontsize=fS)
plt.grid(True)
plt.rcParams["font.family"] = "Times New Roman"
plt.axis([-1, 1, -1, 1])
# plt.savefig('../images/traj_result.pdf')

In [ ]:
fS = 15
plt.figure(0,figsize=(15,7))
plot_traj_set(xbar,ubar,c_list,H_list,Qbar,xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt,flag_label=True)
for xsam_e in xsamp :
    plt.plot(xsam_e[:,0], xsam_e[:,1],'o',markersize=3.0, linewidth=1.0,alpha=1.0,color='tab:purple')
#     plt.plot(xsam_e[1,0], xsam_e[1,1],'o',markersize=3.0, linewidth=1.0,alpha=1.0,color='tab:purple')
plt.plot(1e3,1e3,'o-',color='tab:purple',label='samples')
# plt.plot(xnom[:,0],xnom[:,1],'-.',color='tab:brown',label='traj w/o funnel')
# plt.legend(fontsize=fS)
plt.grid(True)
plt.rcParams["font.family"] = "Times New Roman"
plt.axis([7, 9, 3, 5])
# plt.savefig('../images/traj_result.pdf')

# Search the case of violating the invariance

In [ ]:
from scipy.integrate import solve_ivp
def dfdt(t,x,u,w) :
    return np.squeeze(myModel.forward_uncertain(x,u,w))

num_test = 10000
for i in range(num_test) :
    z = np.random.randn(ix)
    z = z / np.linalg.norm(z)
    x_s = xbar[0] + scipy.linalg.sqrtm(Qbar[0])@z
    
    u = ubar[0] + Kbar[0]@(x_s-xbar[0])
    
    z = np.random.randn(iw)
    w = z / np.linalg.norm(z)
#     w = np.array([1,0])
    
    sol = solve_ivp(dfdt,(0,delT),x_s,args=(u,w),max_step=delT/10,method='RK45',rtol=1e-6,atol=1e-10)
    
    xnew = sol.y[:,-1]
    radi_next = (xnew-xbar[1]).T@np.linalg.inv(Qbar[1])@(xnew-xbar[1])
    if radi_next > 1 :
        print("found!",radi_next)
        break
    if i == num_test - 1 :
        print("no case")
        
    
    

In [ ]:
fS = 15
plt.figure(0,figsize=(15,7))
plot_traj_set(xbar[0:2],ubar[0:2],c_list,H_list,Qbar[0:2],xi=xi,xf=xf,Qi=Qini,Qf=Qf,plt=plt,flag_label=True)
plt.plot(xnew[0], xnew[1],'o',markersize=3.0, linewidth=1.0,alpha=1.0,color='tab:purple')
plt.plot(1e3,1e3,'o-',color='tab:purple',label='samples')
plt.grid(True)
plt.rcParams["font.family"] = "Times New Roman"
plt.axis([-1, 1, -1, 1])